# Huấn luyện có giám sát với SFTTrainer

Bài học này se dạy bạn các huấn luyện mô hình `HuggingFaceTB/SmolLM2-135M` bằng `SFTTrainer` trong thư viện `trl`.  Các cell trong notebook này sẽ chạy và huấn luyện mô hình. Bạn có thể chọn độ khó bằng cách thử nghiệm với các bộ dữ liệu khác nhau.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bài tập: Fine-Tuning SmolLM2 với SFTTrainer</h2>
    <p>Chọn một bộ dự liệu từ Hugging Face hub và huấn luyện một mô hình trên bộ dữ liệu đó. </p> 
    <p><b>Các bài tập</b></p>
    <p>🐢 Sử dụng bộ dữ liệu `HuggingFaceTB/smoltalk`</p>
    <p>🐕 Thử nghiệm với bộ dữ liệu `bigcode/the-stack-smol` và huấn luyện một mô hình sinh code trên tập con cụ thể `data/python`.</p>
    <p>🦁 Chọn một bộ dữ liệu liên quan đến một lĩnh vực mà bạn quan tâm</p>
</div>

In [ ]:
# Cài đặt các thư viện cần thiết
# !pip install transformers datasets trl huggingface_hub

# Đăng nhập vào Hugging Face
from huggingface_hub import login

login()

# Để thuận tiện, bạn có thể tạo một biến môi trường chứa `token hub` của bạn dưới dạng HF_TOKEN

In [ ]:
# Các thư viện cần thiết
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Tải mô hình và tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Thiết lập định dạng chat
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Đặt tên cho mô hình huấn luyện để lưu &/ tải lên
finetune_name = "SmolLM2-FT-MyDataset"
finetune_tags = ["smol-course", "module_1"]

# Sinh văn bản với Mô hình gốc

Ở đây chúng ta sẽ thử nghiệm mô hình gốc chưa được huấn luyện trên định dạng chat.

In [ ]:
# Kiểm tra mô hình gốc trước khi huấn luyện
prompt = "Write a haiku about programming"

# Định dạng
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Tạo phản hồi từ mô hình
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

## Chuẩn bị dữ liệu

Chúng ta sẽ tải một bộ dữ liệu mẫu và định dạng nó cho việc huấn luyện. Bộ dữ liệu cần được cấu trúc với các cặp đầu vào - đầu ra, trong đó mỗi đầu vào là một chỉ thị và đầu ra là phản hồi mong đợi từ mô hình.

**TRL sẽ định dạng các tin nhắn đầu vào dựa trên định dạng chat của mô hình** Chúng cần được biểu diễn dưới dạng danh sách các từ điển với các khóa: `role` và `content`.

**Ví dụ:**
```sh
[
  {"role": "user", "content": "Hello, how are you?"},
  {"role": "assistant", "content": "I'm doing well, thank you! How can I assist you today?",},
]
```

In [ ]:
# Tải dữ liệu mẫu
from datasets import load_dataset

# TODO: Tải bộ dữ liệu thông qua việc điều chỉnh các tham số path và name
ds = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")

In [ ]:
# TODO: 🦁 Nếu dataset của bạn không ở định dạng mà TRL có thể chuyển đổi thành định dạng chat, bạn sẽ cần xử lý nó.
# Tham khảo [Định dạng Chat](../chat_templates.md)

## Điều chỉnh SFTTrainer

Điều chỉnh `SFTTrainer` với các tham số khác nhau giúp điều khiển quá trình huấn luyện trở nên hiệu quả hơn. Các thông số bao gồm
- Số bước huấn luyện (steps)
- Kích thước batch (batch size)
- Tốc độ học (learning rate)
- Chiến lược đánh giá mô hình (evaluation strategy)

Ngoài ra, còn rất nhiều thông số khác, bạn có thể tham khảo thêm ở [SFTTrainer](https://huggingface.co/docs/trl/sft_trainer)

Điều chỉnh các tham số này dựa trên yêu cầu cụ thể và tài nguyên tính toán của bạn.

In [ ]:
# Điều chỉnh SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Điều chỉnh dựa trên kích thước dataset và thời lượng huấn luyện mong muốn
    per_device_train_batch_size=4,  # Đặt theo dung lượng bộ nhớ GPU của bạn
    learning_rate=5e-5,  # Phổ biến cho quá trình huấn luyện có giám sát
    logging_steps=10,  # Tần suất ghi log các metrics huấn luyện
    save_steps=100,  # Tần suất lưu các checkpoint mô hình
    evaluation_strategy="steps",  # Đánh giá mô hình theo các khoảng thời gian
    eval_steps=50,  # Tần suất đánh giá
    use_mps_device=(
        True if device == "mps" else False
    ),  # Sử dụng MPS cho huấn luyện độ chính xác hỗn hợp
    hub_model_id=finetune_name,  # Đặt tên cho mô hình của bạn
)

# Khởi tạo SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    tokenizer=tokenizer,
    eval_dataset=ds["test"],
)


# TODO: 🦁 🐕 căn chỉnh các tham số SFTTrainer với bộ dữ liệu bạn đã chọn.
# Ví dụ, nếu bạn đang sử dụng bộ `bigcode/the-stack-smol`, bạn sẽ cần chọn cột `content`

## Huấn luyện Mô hình

Với trainer đã được điều chỉnh, chúng ta có thể tiến hành huấn luyện mô hình. Quá trình huấn luyện sẽ bao gồm
- Lặp qua bộ dữ liệu
- Tính toán loss
- Cập nhật các tham số của mô hình để giảm thiểu loss này.

In [ ]:
# Huấn luyện mô hình
trainer.train()

# Lưu mô hình
trainer.save_model(f"./{finetune_name}")

In [ ]:
# Đưa mô hình lên Hugging Face Hub
trainer.push_to_hub(tags=finetune_tags)

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bài tập thêm: Sinh văn bản với mô hình vừa được huấn luyện</h2>
    <p>🐕 Sử dụng mô hình đã được huấn luyện để sinh ra phản hồi, giống như với ví dụ ban đầu.</p>
</div>

In [ ]:
# Kiểm tra mô hình đã được huấn luyện trên cùng một yêu cầu

# Kiểm tra mô hình gốc trước khi huấn luyện
prompt = "Write a haiku about programming"

# Định dạng chat
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Sinh phản hồi từ mô hình
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

# TODO: sử dụng mô hình đã được huấn luyện để sinh phản hồi, giống như với ví dụ.

## 💐 Chúc mừng bạn. Bạn đã hoàn thành!

Bài tập này đã cung cấp hướng dẫn từng bước để bạn huấn luyện được mô hình `HuggingFaceTB/SmolLM2-135M` sử dụng `SFTTrainer`. Bằng cách làm theo các bước này, bạn có thể điều chỉnh mô hình để thực hiện các tác vụ cụ thể hiệu quả hơn. Nếu bạn muốn tiếp tục làm việc với khóa học này, đây là một số bước bạn có thể thử:

- Thử notebook này ở mức độ khó hơn
- Review PR của học viên khác
- Cải thiện tài liệu khóa học thông qua Issue hoặc PR.